In [15]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

from tools.torch_lib import *

from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import copy
from torchmetrics.regression import MeanAbsolutePercentageError

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')
device = cpu

if torch.cuda.is_available():
    device = gpu
    # The flag below controls whether to allow TF32 on matmul. This flag defaults to False
    # in PyTorch 1.12 and later.
    torch.backends.cuda.matmul.allow_tf32 = True
    # The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
    torch.backends.cudnn.allow_tf32 = True

print(device)

cuda


### Load dataframe

In [18]:
dataset_dir = "dataset/"
dataset_file_name = "pz_2a.csv"
plots_dir = "plots/"
test_plots_dir = "test_plots/"

In [19]:
df = pd.read_csv(dataset_dir + dataset_file_name)
df.head()

,ro_well,ro_formation,rok,r_well,lambda1
0,0.01,0.100000,0.119654,0.04,1.0
1,0.01,0.158489,0.204781,0.04,1.0
2,0.01,0.251189,0.350298,0.04,1.0
3,0.01,0.398107,0.590140,0.04,1.0
4,0.01,0.630957,0.967633,0.04,1.0


In [20]:
df.columns

Index(['ro_well', 'ro_formation', 'rok', 'r_well', 'lambda1'], dtype='object')

In [21]:
# print attribute's min max

In [22]:
for column in df.columns:
    print(f"{column}: min={df[column].min()} max={df[column].max()}")

ro_well: min=0.01 max=1000.0
ro_formation: min=0.1 max=10000.0
rok: min=0.0944038 max=21496.2
r_well: min=0.04 max=0.2
lambda1: min=1.0 max=5.0


In [23]:
# attributes in logarithmic scale:
for column in df.columns:
    if column == 'd_well':
        continue
    print(f"{column}: min={np.log(df[column].min())} max={np.log(df[column].max())}")

ro_well: min=-4.605170185988091 max=6.907755278982137
ro_formation: min=-2.3025850929940455 max=9.210340371976184
rok: min=-2.360173952403574 max=9.975631454308614
r_well: min=-3.2188758248682006 max=-1.6094379124341003
lambda1: min=0.0 max=1.6094379124341003


### Add dataframe transforms

In [24]:
inputs = np.array(['ro_well', 'ro_formation', 'r_well', 'lambda1'])
outputs = np.array(['rok']) # 'A02M01N' dropped

In [25]:
logarithmic_columns = ['ro_formation', 'ro_well']
# normalize data ('min/max' normalization):
interval_th = [-1, 1]     # normalization interval for 'th' activation function
interval_leaky_relu = [-0.2, 1]
interval_sigmoid = [0, 1] # normalization interval for 'sigmoid' activation function
normalize_interval = interval_sigmoid

attributes_transform_dict = {}
df_transformed = df.copy()

# transform output attributes:
for output_attr in outputs:
    attr_transformer = attributes_transform_dict[output_attr] = AttributeTransformer(df_transformed[output_attr].to_numpy())

    # logarithmic transform
    forward, backward = np.log, np.exp
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)
    # scaling transform
    #forward, backward = get_standard_scaler_transform(attr_transformer.data)
    #df_transformed[output_attr] = attr_transformer.transform(forward, backward)
    # # normalize transform
    forward, backward = get_normalize_transforms(attr_transformer.data, normalize_interval)
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)

# logarithm resistance:
for col in logarithmic_columns:
    if col in outputs:
        continue
    df_transformed[col] = df_transformed[col].apply(np.log)

# add normalization
for attribute in df_transformed.columns:
    if attribute in outputs:
        continue
    transform, _ = get_normalize_transforms(df_transformed[attribute].to_numpy(), normalize_interval)
    #transform, _ = get_standard_scaler_transform(df_transformed[attribute].to_numpy())  # use scaling instead of min-max norm
    df_transformed[attribute] = transform(df_transformed[attribute].to_numpy())

df_transformed

,ro_well,ro_formation,rok,r_well,lambda1
0,0.0,0.00,0.019214,0.0,0.0
1,0.0,0.04,0.062773,0.0,0.0
2,0.0,0.08,0.106292,0.0,0.0
3,0.0,0.12,0.148574,0.0,0.0
4,0.0,0.16,0.188660,0.0,0.0
...,...,...,...,...,...
487573,1.0,0.84,0.828126,1.0,1.0
487574,1.0,0.88,0.865705,1.0,1.0
487575,1.0,0.92,0.901307,1.0,1.0
487576,1.0,0.96,0.934697,1.0,1.0


In [26]:
def print_inference_statistic(attributes, df_):
    means = []
    stds = []
    mins = []
    maxes = []

    for column in attributes:
        col_data = df_[column].to_numpy()

        if column in logarithmic_columns or column in outputs:
            col_data = np.log(col_data)  # first transform - log

        # col_mean = np.mean(col_data)
        # col_std = np.std(col_data)

        # means.append(col_mean)
        # stds.append(col_std)
        #
        # col_data = (col_data - col_mean) / col_std

        mins.append(np.min(col_data))
        maxes.append(np.max(col_data))

    # print(f"means={means}")
    # print(f"stds={stds}")
    print(f"mins={mins}")
    print(f"maxes={maxes}")

In [27]:
print_inference_statistic(inputs, df)

mins=[-4.605170185988091, -2.3025850929940455, 0.04, 1.0]
maxes=[6.907755278982137, 9.210340371976184, 0.2, 5.0]


In [28]:
print_inference_statistic(outputs, df)

mins=[-2.360173952403574]
maxes=[9.975631454308614]


### Build Datasets and create dataloaders

In [12]:
class SimpleDataset(Dataset):
    def __init__(self, df_, inputs, outputs, device):
        self.df = df_
        self.inputs = torch.from_numpy(df_[inputs].to_numpy()).float().to(device)
        self.outputs = torch.from_numpy(df_[outputs].to_numpy()).float().to(device)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        item, label = self.inputs[idx], self.outputs[idx]

        return item, label


In [13]:
batch_size = 20000

train_df, test_df = train_test_split(df_transformed, shuffle=True, test_size=0.3)
test_df, validation_df = train_test_split(test_df, shuffle=True, test_size=0.33)

train_dataset = SimpleDataset(train_df, inputs, outputs, device)
test_dataset = SimpleDataset(test_df, inputs, outputs, device)
validation_dataset = SimpleDataset(validation_df, inputs, outputs, device)
full_dataset = SimpleDataset(df_transformed, inputs, outputs, device)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
full_dataset_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

### Build models

In [14]:
class WeightedMAE(nn.Module):
    def __init__(self, weights):
        super(WeightedMAE, self).__init__()
        self.mae = nn.L1Loss()
        self.weights = weights

    def forward(self, inputs, targets):
        weighted_inputs = inputs * self.weights

        return self.mae(weighted_inputs, targets)

    def to(self, device):
        super().to(device)
        self.weights = self.weights.to(device)


class LinearModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        module_list = []
        for i in range(layers_count - 1):
            module_list.append(nn.Linear(layers_dims[i], layers_dims[i + 1]))
        module_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))

        activations_list = []
        for i in range(layers_count):
            activations_list.append(activations[act_str_list[i]])

        self.linears = nn.ModuleList(module_list)
        self.activations = nn.ModuleList(activations_list)

    def forward(self, x):
        y = x

        for lin, act in zip(self.linears, self.activations):
            y = lin(y)
            y = act(y)

        return y


class LinearLNormModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        linears_list = []
        layers_norm_list = []

        for i in range(layers_count - 1):
            in_features, out_features = layers_dims[i], layers_dims[i + 1]
            linears_list.append(nn.Linear(in_features, out_features))
            layers_norm_list.append(nn.LayerNorm(out_features))
        # add last layer
        linears_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))
        layers_norm_list.append(nn.LayerNorm(output_dim))

        self.linears = nn.ModuleList(linears_list)
        self.activations = nn.ModuleList([activations[act_str_list[i]] for i in range(len(act_str_list))])
        self.layer_normalizations = nn.ModuleList(layers_norm_list)

    def forward(self, x):
        y = x

        for lin, act, norm in zip(self.linears, self.activations, self.layer_normalizations):
            y = lin(y)
            y = norm(y)
            y = act(y)

        return y


# add batch normalization
class LinearBNormModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        linears_list = []
        batch_norm_list = []

        for i in range(layers_count - 1):
            in_features, out_features = layers_dims[i], layers_dims[i + 1]
            linears_list.append(nn.Linear(in_features, out_features))
            batch_norm_list.append(nn.BatchNorm1d(out_features))

        linears_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))
        batch_norm_list.append(nn.BatchNorm1d(output_dim))

        activations_list = []
        for i in range(layers_count):
            activations_list.append(activations[act_str_list[i]])

        self.linears = nn.ModuleList(linears_list)
        self.activations = nn.ModuleList(activations_list)
        self.batch_normalizations = nn.ModuleList(batch_norm_list)

    def forward(self, x):
        y = x

        for lin, act, norm in zip(self.linears, self.activations, self.batch_normalizations):
            y = lin(y)
            y = norm(y)
            y = act(y)

        return y

### Train model

In [15]:
layers_dims = [len(inputs), 40, 150, 1500, 150, 10]
layers_count = len(layers_dims)
activations_string_list = ['leaky-relu' for i in range(layers_count)]
#activations_string_list[-1] = 'sigmoid'

linear_model = LinearModel(layers_dims, activations_string_list, len(outputs)).to(device)
#linear_bn_model = LinearBNormModel(layers_dims, activations_string_list, len(outputs)).to(device)
#linear_ln_model = LinearLNormModel(layers_dims, activations_string_list, len(outputs)).to(device)

model = linear_model
model_name = "linear_model"
linear_model

LinearModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=150, bias=True)
    (2): Linear(in_features=150, out_features=1500, bias=True)
    (3): Linear(in_features=1500, out_features=150, bias=True)
    (4): Linear(in_features=150, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
  (activations): ModuleList(
    (0-5): 6 x LeakyReLU(negative_slope=0.01)
  )
)

In [16]:
learning_rate = 0.0001
epoch_count = 200

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

#loss_function = WeightedMAE(torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0], dtype=float))
loss_function = nn.L1Loss()

In [17]:
epoch_validation = True
train_loss_threshold = 0.0003

train_loss_list, validation_loss_list = train_model(epoch_count, model, optimizer, loss_function, train_loader, validation_loader, True, train_loss_threshold)
plot_loss(train_loss_list, "train loss")

Epoch: 0; train loss=0.246051; validation loss=0.209986
Epoch: 1; train loss=0.185595; validation loss=0.171099
Epoch: 2; train loss=0.155382; validation loss=0.136974
Epoch: 3; train loss=0.110655; validation loss=0.080052
Epoch: 4; train loss=0.065023; validation loss=0.053110
Epoch: 5; train loss=0.045475; validation loss=0.039157
Epoch: 6; train loss=0.034947; validation loss=0.030494
Epoch: 7; train loss=0.027528; validation loss=0.025657
Epoch: 8; train loss=0.022680; validation loss=0.020428
Epoch: 9; train loss=0.018764; validation loss=0.016958
Epoch: 10; train loss=0.015508; validation loss=0.013455
Epoch: 11; train loss=0.012604; validation loss=0.011841
Epoch: 12; train loss=0.010632; validation loss=0.009801
Epoch: 13; train loss=0.008804; validation loss=0.008763
Epoch: 14; train loss=0.007942; validation loss=0.007363
Epoch: 15; train loss=0.006936; validation loss=0.005948
Epoch: 16; train loss=0.006156; validation loss=0.006437
Epoch: 17; train loss=0.005584; validatio

KeyboardInterrupt: 

In [ ]:
test_loss = test_loop(test_loader, model, loss_function)
print(f"test loss={test_loss}")

In [ ]:
plot_loss(validation_loss_list, "test loss")

### Plot predictions

In [ ]:
for _, (X, y) in enumerate(train_loader):
    print(model(X))
    break

In [ ]:
# plot_predictions(outputs, full_dataset_loader, linear_model)

In [ ]:
#plot_actual_predictions(outputs, full_inference_dataset_loader, linear_model, attributes_transform_dict, df)

In [18]:
model = torch.load("saved_models/" + "linear_model0_0007086.pth")
model.to(device)

LinearModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=120, bias=True)
    (2): Linear(in_features=120, out_features=1200, bias=True)
    (3): Linear(in_features=1200, out_features=120, bias=True)
    (4): Linear(in_features=120, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
  (activations): ModuleList(
    (0-5): 6 x LeakyReLU(negative_slope=0.01)
  )
)

In [21]:
plot_relative_errors(outputs, full_dataset_loader, model, attributes_transform_dict,
                     df, 0.01, device, plots_dir, mode='default+hist', bin_count=100)

In [ ]:
# plot test predictions:
plot_relative_errors(outputs, test_loader, model, attributes_transform_dict,
                     df, 0.01, device, plots_dir + test_plots_dir, mode='default+hist', bin_count=100)

#### check predictions manually

In [ ]:
predictor = Predictor(full_dataset_loader, df, attributes_transform_dict, model, inputs, outputs)
predictions_dict, actuals_dict = predictor.predict(device)

In [ ]:
def compare_prediction(idx: int, prediction_dict, actuals_dict, attribute):
    predicted = prediction_dict[attribute][idx]
    actual = actuals_dict[attribute][idx]
    relative_error = abs(actual - predicted) / actual
    print(f"{idx}: predicted={predicted}; actual={actual}; relative error={relative_error}")

### Save model

In [ ]:
model.to(cpu)    # attach model to cpu before scripting and saving to prevent cuda meta information saved
scripted_model = torch.jit.script(model)
models_dir = "saved_models/"
model_file_name = models_dir + model_name + str(round(test_loss, 7)).replace('.', '_')

scripted_model.save(model_file_name + ".pt") # save torch script model which compatible with pytorch c++ api
torch.save(model, model_file_name + ".pth")   # save model in python services specific format

# attach model back to device:
model.to(device)

In [ ]:
scripted_model(torch.tensor([0.6, 0.362372, 0.04]))

In [ ]:
model(torch.tensor([0.6, 0.362372, 0.04], device=device))

### Check non linearly

In [ ]:
model = torch.load("saved_models/" + "linear_model0_0007086.pth")
model.to(device)

In [ ]:
batch_idx = 3
layer_idx = 0
for i in range(batch_idx + 1):
    batch = next(iter(full_dataset_loader))
batch = batch[0]

plot_layer_distribution(model, layer_idx, batch)

In [ ]:
batch

### Check classic ML models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


class LinearRegressionModel(object):
    def __init__(self, n_jobs:int = 10):
        self.n_jobs = n_jobs
        self.model = LinearRegression(n_jobs=n_jobs)
        self.degree = 2

    def fit(self, train_df_, inputs_, outputs_, degree: int = 2):
        train_data_X = train_df_[inputs_].to_numpy()
        train_data_Y = train_df_[outputs_].to_numpy()

        polynomial_transform = PolynomialFeatures(degree=degree)
        train_data_X_transformed = polynomial_transform.fit_transform(train_data_X)

        self.model.fit(train_data_X_transformed, train_data_Y)
        self.degree = degree

    def predict(self, X_data, do_poly_transform: bool = True):
        input_X = X_data

        if do_poly_transform:
            polynomial_transform = PolynomialFeatures(degree=self.degree)
            input_X = polynomial_transform.fit_transform(input_X)

        return self.model.predict(input_X)

    def test(self, X_data, Y_actual, do_poly_transform: bool = True):
        predicted = self.predict(X_data, do_poly_transform)
        loss_ = torch.nn.L1Loss()

        return loss_(torch.tensor(predicted), torch.tensor(Y_actual))


In [ ]:
def plot_relative_error_arrays(actuals, predictions, outputs,
                                   transform_dict, model_name: str,
                                   threshold: float = 0.05, width: int = 10000,
                                   height: int = 500, with_shader=True):
    for i in range(len(outputs)):
        output_attr = outputs[i]
        transformer = transform_dict[output_attr]

        # transform predictions back:
        prediction = predictions[:, i]
        transformer.set_data(prediction)
        final_prediction = torch.from_numpy(transformer.transform_backward())

        # get actual data from dataloader
        actual = actuals[:, i]
        transformer.set_data(actual)
        actual = torch.from_numpy(transformer.transform_backward())

        # plot graphic
        if with_shader:
            fig = plot_relative_error_shader(actual, final_prediction, threshold, 'relative error ' + output_attr, width, height)
        else:
            fig = plot_relative_error(actual, final_prediction, threshold, 'relative error ' + output_attr, width, height)
        fig.show('browser')
        fig.write_image(plots_dir + "err_" + model_name + "_" + output_attr + ".pdf")

In [ ]:
linear_regression_model = LinearRegressionModel()
linear_regression_model.fit(train_df, inputs, outputs, degree=10)

In [ ]:
loss = linear_regression_model.test(df_transformed[inputs].to_numpy(), df_transformed[outputs].to_numpy())
print(loss)

In [ ]:
predictions = linear_regression_model.predict(df_transformed[inputs].to_numpy())
predictions.shape

In [ ]:
plot_relative_error_arrays(df_transformed[outputs].to_numpy(), predictions, outputs,
                           attributes_transform_dict, "linear_regression", threshold=0.01)